<a href="https://colab.research.google.com/github/HillaryKoros/geeStuff/blob/main/Kenya_LST_MODIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !pip install geemap

In [4]:
import ee
import geemap
import os

In [6]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [7]:
# // Import country boundaries feature collection.
dataset = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017');

# // Apply filter where country name equals Uganda.
kenyaBorder = dataset.filter(ee.Filter.eq('country_na', 'Kenya'));

# // Add Uganda outline to the Map as a layer.
Map.centerObject(kenyaBorder, 6);
Map.addLayer(kenyaBorder);
Map

Map(bottom=754.0, center=[0.5578063517900957, 37.84559934447033], controls=(WidgetControl(options=['position',…

In [9]:
# Add Earth Engine dataset
modisLST = ee.ImageCollection('MODIS/MOD11A2');
start = ee.Date('2015-01-01');
end = ee.Date('2015-12-31');
dateRange = ee.DateRange(start, end);
# // date range.
mod11a2 = modisLST.filterDate(dateRange);

# // Select only the 1km day LST data band.
modLSTday = mod11a2.select('LST_Day_1km');

def func_psu(img):
  return img \
    .multiply(0.02) \
    .subtract(273.15) \
    .copyProperties(img, ['system:time_start'])

modLSTc = modLSTday.map(func_psu)

clippedLSTc = modLSTc.mean().clip(kenyaBorder);

# // Add clipped image layer to the map.
Map.addLayer(clippedLSTc, {'min': 18, 'max': 45,
  'palette': ['blue', 'limegreen', 'yellow', 'darkorange', 'red']},
  'Mean temperature, 2015');
Map

Map(bottom=8467.0, center=[0.5578063517900957, 37.84559934447033], controls=(WidgetControl(options=['position'…

In [10]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_lst_stats = os.path.join(out_dir, 'lst_stats.csv')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(clippedLSTc, kenyaBorder, out_lst_stats, statistics_type='MEAN', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /root/Downloads/lst_stats.csv


In [12]:
geemap.create_download_link(out_lst_stats)